In [2]:
## change the working directory to the main folder
import os

os.chdir("/Users/bhargobdeka/Desktop/Projects/GenAI/bim_ir_chat")


In [3]:
## check the current working directory
%pwd

'/Users/bhargobdeka/Desktop/Projects/GenAI/bim_ir_chat'

In [ ]:
## check if .env has all the api keys and load them

from dotenv import dotenv_values
# dotenv_values(".env")


In [4]:

## Getting the api keys from the .env file

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Langsmith Tracing
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

# print(os.environ['LANGCHAIN_API_KEY'])

In [5]:
from modules.llm.llm_settings import LLMSettings

# check the available LLM models -- can include more models
LLMSettings.MODEL_OPTIONS

['gpt-4', 'gpt-3.5-turbo', 'claude-3-opus-20240229']

In [18]:
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import get_openai_callback

from modules.llm.llm_settings import LLMSettings

model = 'gpt-4'
llm_settings = LLMSettings(model)



In [19]:
## Loading LLM model
llm = llm_settings.get_correct_langchain_llm()
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x137e3fd00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x137c6cf40>, model_name='gpt-4', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [9]:
## Retriever model
from modules.llm.tools import get_retriever

retriever = get_retriever("documents/nbc2020.pdf")

In [16]:
## Prompt Template
from langchain.prompts.prompt import PromptTemplate

qa_template = """
    
        You are a helpful AI assistant named Alago. The user provides you with a file whose content is represented by the following context parts (delimited by <context></context>). 
        Use these to answer the question at the end.
        If there is any history of previous conversations, use it to answer (delimited by <history></history>)
        The files deal with topics from the construction industry. You can assume that the user is a civil engineer or architect.
        If you don't know the answer, just say that you don't know. Do NOT try to make up an answer.
        If the question is not related to the context, politely respond that you only answer questions related to the context.
        Provide as detailed an answer as possible.

        <context>
        {context}
        </context>
        
        <history>
        {chat_history}
        </history>
        
        <question>
        {question}
        </question>
    
    """

prompt = PromptTemplate(template=qa_template, input_variables=["context", "chat_history", "question"])


In [13]:
## Create Conversational Retrieval chain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# define memory to store chat history
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True, output_key='answer')


In [20]:
## creating chain
chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                              chain_type="stuff",
                                              retriever=retriever,
                                              memory=memory,
                                              combine_docs_chain_kwargs={"prompt": prompt},
                                              output_key='answer',
                                              verbose=True
                                              )

In [21]:
# chain_input
query = "What are the design requirements of a structural design?"

chain_input = {"question": query, "chat_history": []}

result = chain(chain_input)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    
        You are a helpful AI assistant named Alago. The user provides you with a file whose content is represented by the following context parts (delimited by <context></context>). 
        Use these to answer the question at the end.
        If there is any history of previous conversations, use it to answer (delimited by <history></history>)
        The files deal with topics from the construction industry. You can assume that the user is a civil engineer or architect.
        If you don't know the answer, just say that you don't know. Do NOT try to make up an answer.
        If the question is not related to the context, politely respond that you only answer questions related to the context.
        Provide as detailed an answer as possible.

        <context>
        A-5.1.4.1.
Division B
A-5.1.4.1.
Application of Structural Design to Other Building Elements. Part 4, as 

In [24]:
print(result['answer'])

The design requirements of a structural design, as per the context, include:

1) Buildings and their structural members and connections, including formwork and falsework, should be designed to have sufficient structural capacity and structural integrity to safely and effectively resist all loads, effects of loads and influences that may reasonably be expected, having regard to the expected service life of buildings. They should satisfy the requirements of this Section.

2) Buildings and their structural members should be designed for serviceability, in accordance with Articles 4.1.3.4., 4.1.3.5. and 4.1.3.6.

In addition, structural drawings and related documents submitted with the application to build should indicate:

a) The name and address of the person responsible for the structural design,
b) The date of issue of the Code and standards to which the design conforms,
c) The dimensions, location and size of all structural members in sufficient detail to enable the design to be check

In [26]:
query = "What are the main points to consider while designing a structural member for serviceability?"

history = result['chat_history']
chain_input = {"question": query, "chat_history": history}

result = chain(chain_input)

result['answer']



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the design requirements of a structural design?
Assistant: The design requirements of a structural design, as per the context, include:

1) Buildings and their structural members and connections, including formwork and falsework, should be designed to have sufficient structural capacity and structural integrity to safely and effectively resist all loads, effects of loads and influences that may reasonably be expected, having regard to the expected service life of buildings. They should satisfy the requirements of this Section.

2) Buildings and their structural members should be designed for serviceability, in accordance with Articles 4.1.3.4., 4.1.3.5. and 4.1.3.6.

In addition, structural drawings and related documents submitted with the application 

'When designing a structural member for serviceability, the following key factors should be taken into account:\n\n1) The building and its structural components should be checked for serviceability limit states under the effect of service loads for serviceability criteria specified or recommended in Articles 4.1.3.5. and 4.1.3.6. and in the standards listed in Section 4.3.\n\n2) The effect of service loads on the serviceability limit states should be determined in accordance with this Article and the load combinations listed in Table 4.1.3.4., the applicable combination being that which results in the most critical effect.\n\n3) Other load combinations that must also be considered are the principal loads acting with the companion loads taken as zero.\n\n4) In proportioning structural members to limit serviceability problems resulting from deflections, consideration should be given to the intended use of the building or member, limiting damage to non-structural members made of materials

In [27]:
print(result['answer'])

When designing a structural member for serviceability, the following key factors should be taken into account:

1) The building and its structural components should be checked for serviceability limit states under the effect of service loads for serviceability criteria specified or recommended in Articles 4.1.3.5. and 4.1.3.6. and in the standards listed in Section 4.3.

2) The effect of service loads on the serviceability limit states should be determined in accordance with this Article and the load combinations listed in Table 4.1.3.4., the applicable combination being that which results in the most critical effect.

3) Other load combinations that must also be considered are the principal loads acting with the companion loads taken as zero.

4) In proportioning structural members to limit serviceability problems resulting from deflections, consideration should be given to the intended use of the building or member, limiting damage to non-structural members made of materials whose ph

In [28]:
query = "What was my first question?"

history = result['chat_history']
chain_input = {"question": query, "chat_history": history}

result = chain(chain_input)

print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the design requirements of a structural design?
Assistant: The design requirements of a structural design, as per the context, include:

1) Buildings and their structural members and connections, including formwork and falsework, should be designed to have sufficient structural capacity and structural integrity to safely and effectively resist all loads, effects of loads and influences that may reasonably be expected, having regard to the expected service life of buildings. They should satisfy the requirements of this Section.

2) Buildings and their structural members should be designed for serviceability, in accordance with Articles 4.1.3.4., 4.1.3.5. and 4.1.3.6.

In addition, structural drawings and related documents submitted with the application 

In [29]:
query = "What was my last question?"

history = result['chat_history']
chain_input = {"question": query, "chat_history": history}

result = chain(chain_input)

print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the design requirements of a structural design?
Assistant: The design requirements of a structural design, as per the context, include:

1) Buildings and their structural members and connections, including formwork and falsework, should be designed to have sufficient structural capacity and structural integrity to safely and effectively resist all loads, effects of loads and influences that may reasonably be expected, having regard to the expected service life of buildings. They should satisfy the requirements of this Section.

2) Buildings and their structural members should be designed for serviceability, in accordance with Articles 4.1.3.4., 4.1.3.5. and 4.1.3.6.

In addition, structural drawings and related documents submitted with the application 